In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import numpy as np
import scipy
import time
from PIL import Image,ImageOps
from keras.datasets import mnist
from keras import layers
import keras
from keras.layers import Dropout
from matplotlib import pyplot as plt
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical
from keras import initializers
from keras import optimizers
from matplotlib import pyplot
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras.layers.normalization import BatchNormalization
from array import array
import cv2
import os
import random
from google.colab.patches import cv2_imshow

In [ ]:
new_shape = (28,28)
def load_images_from_folder(folder,images,y,i):
  
    for filename in os.listdir(folder):
        img=cv2.imread(os.path.join(folder,filename))
        #img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #img = Image.open(os.path.join(folder,filename))
        #print(type(img))
        if img is not None:
            #print(img.shape)
            #plt.imshow(img)
            img = cv2.resize(img,new_shape)
            images.append(img) 
            
            im=img.copy()
            mean=0
            st=0.7
            gauss = np.random.normal(mean,st,im.shape)
            gauss = gauss.astype('uint8')
            im = cv2.add(im,gauss)
            images.append(im)

            # ima=img.copy()
            # value = np.random.choice(np.array([-50, -40, -30, 30, 40, 50]))
            # hsv = cv2.cvtColor(ima, cv2.COLOR_BGR2HSV)
            # h, s, v = cv2.split(hsv)
            # if value >= 0:
            #   lim = 255 - value
            #   v[v > lim] = 255
            #   v[v <= lim] += value
            # else:
            #   lim = np.absolute(value)
            #   v[v < lim] = 0
            #   v[v >= lim] -= np.absolute(value)

            # final_hsv = cv2.merge((h, s, v))
            # ima = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
          

            # value = np.random.choice(np.array([-50, -40, -30, 30, 40, 50]))
            # hsv = cv2.cvtColor(ima, cv2.COLOR_BGR2HSV)
            # h, s, v = cv2.split(hsv)
            # if value >= 0:
            #   lim = 255 - value
            #   s[s > lim] = 255
            #   s[s <= lim] += value
            # else:
            #   lim = np.absolute(value)
            #   s[s < lim] = 0
            #   s[s >= lim] -= np.absolute(value)

            # final_hsv = cv2.merge((h, s, v))
            # ima = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)

            #images.append(ima)

            im=img.copy()
            brightness = 10
            contrast = random.randint(40, 100)
            dummy = np.int16(im)
            dummy = dummy * (contrast/127+1) - contrast + brightness
            dummy = np.clip(dummy, 0, 255)
            im = np.uint8(dummy)
            images.append(im)
            # y.append(i-1)
            y.append(i-1)
            y.append(i-1)
            y.append(i-1)
            #i=i+1
    #print(i)
    return images,y

X = []
y=[]
for i in range(1,11):
  #print(i)
  if (i<10):
    folder="/content/gdrive/MyDrive/trainPart1/train/Sample00"+str(i)
  else:
    folder="/content/gdrive/MyDrive/trainPart1/train/Sample0"+str(i)
  X,y=load_images_from_folder(folder,X,y,i)
  


In [ ]:
X=np.array(X)
print(X.shape)
y=np.array(y)
print(y.shape)
train_images, test_imagesall, train_labels, test_labelsall = train_test_split(X, y, test_size=0.20, random_state=42)

print('Shape of images is..',train_images.shape)
print('Shape of labels is..',train_labels.shape)

train_images=train_images.reshape(960,28,28,1)
test_imagesall=test_imagesall.reshape(240,28,28,1)

val_images = test_imagesall[0:100,:,:]
val_labels = test_labelsall[0:100]
test_images = test_imagesall[100:,:,:]
test_labels = test_labelsall[100:]


# display sample
print("Shape of training images:",train_images.shape)
print("Shape of training labels:",train_labels.shape)
print("unique labels:",np.unique(train_labels))
print("Shape of val images:",val_images.shape)
print("Shape of val labels:",val_labels.shape)
print("Shape of test images:",test_images.shape)
print("Shape of test labels:",test_labels.shape)

(1200, 28, 28)
(1200,)
Shape of images is.. (960, 28, 28)
Shape of labels is.. (960,)
Shape of training images: (960, 28, 28, 1)
Shape of training labels: (960,)
unique labels: [0 1 2 3 4 5 6 7 8 9]
Shape of val images: (100, 28, 28, 1)
Shape of val labels: (100,)
Shape of test images: (140, 28, 28, 1)
Shape of test labels: (140,)


In [ ]:
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5
val_images = (val_images / 255) - 0.5
print(train_images.shape)
# Flatten the images.

# train_images = train_images.reshape((-1, 784))
# test_images = test_images.reshape((-1, 784))
# val_images = val_images.reshape((-1, 784))

#train_images = train_images.reshape((-1, 784,3))

#test_images = test_images.reshape((-1, 784,3))
#val_images = val_images.reshape((-1, 784,3))


(960, 28, 28, 1)


In [ ]:
# Build the model.
# model = Sequential([
#  Dense(64, activation='relu', input_shape=(784,)),
#  Dense(64, activation='relu'),
#  Dense(10, activation='softmax'),
# ])

# Build the model
# model = Sequential() # create model
# model.add(Dense(64, input_dim=784, trainable=True,activation='relu', use_bias=True, 
#                 kernel_initializer=initializers.he_normal(seed=None))) # hidden 
# model.add(Dropout(0.5))



# # model.add(Dense(64, trainable=True, activation='relu', use_bias=True, 
# #                 kernel_initializer=initializers.he_normal(seed=None))) # hidden layer
# # model.add(Dropout(0.5))

# # model.add(Dense(128, use_bias=False))
# # model.add(BatchNormalization())
# # model.add(Activation("relu"))

# # model.add(Dense(128, use_bias=False))
# # model.add(BatchNormalization())
# # model.add(Activation("relu"))

# model.add(Dense(64, use_bias=False))
# model.add(BatchNormalization())
# model.add(Activation("relu"))

# model.add(Dense(10, trainable=True, activation='softmax')) # output layer
##################
model = Sequential()

model.add(layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
model.add(layers.AveragePooling2D())

model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model.add(layers.AveragePooling2D())

model.add(layers.Flatten())

model.add(layers.Dense(units=120, activation='relu'))

model.add(layers.Dense(units=84, activation='relu'))

model.add(layers.Dense(units=10, activation = 'softmax'))

# model=keras.applications.VGG19(
#     include_top=False,
#     weights=None,
#     input_tensor=None,
#     input_shape=(200,200,3),
#     pooling=None,
#     classes=62,
#     classifier_activation="softmax",
# )

In [ ]:
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

model.compile(
  optimizer=adam,
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

# obtain the summary
#saved_model.summary()



es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 5)
mc = ModelCheckpoint('/content/gdrive/MyDrive/models/task2_pretrain.h5', monitor='val_accuracy', mode='max', 
                     verbose=1, save_best_only=True)
print(train_images.shape)
print(val_images.shape)
# Train the model.
history=model.fit(
  train_images,
  to_categorical(train_labels),
  validation_data=(val_images, to_categorical(val_labels)),  
  epochs=20,
  batch_size=32,
  shuffle = True,
  callbacks=[es,mc],
)

# Evaluate the model.



(960, 28, 28, 1)
(100, 28, 28, 1)
Epoch 1/20
30/30 [==============================] - 33s 19ms/step - loss: 2.3043 - accuracy: 0.1025 - val_loss: 2.2400 - val_accuracy: 0.1800

Epoch 00001: val_accuracy improved from -inf to 0.18000, saving model to /content/gdrive/MyDrive/models/task2_pretrain.h5
Epoch 2/20
30/30 [==============================] - 0s 4ms/step - loss: 2.2174 - accuracy: 0.2212 - val_loss: 2.1090 - val_accuracy: 0.2700

Epoch 00002: val_accuracy improved from 0.18000 to 0.27000, saving model to /content/gdrive/MyDrive/models/task2_pretrain.h5
Epoch 3/20
30/30 [==============================] - 0s 4ms/step - loss: 2.0037 - accuracy: 0.3041 - val_loss: 1.8946 - val_accuracy: 0.4100

Epoch 00003: val_accuracy improved from 0.27000 to 0.41000, saving model to /content/gdrive/MyDrive/models/task2_pretrain.h5
Epoch 4/20
30/30 [==============================] - 0s 4ms/step - loss: 1.6781 - accuracy: 0.4638 - val_loss: 1.6339 - val_accuracy: 0.4700

Epoch 00004: val_accuracy im

In [ ]:
saved_model = load_model('/content/gdrive/MyDrive/models/task2_pretrain.h5')
scores = saved_model.evaluate(
  test_images,
  to_categorical(test_labels)
)

print('Test accuracy:', scores[1])


5/5 [==============================] - 0s 11ms/step - loss: 0.7091 - accuracy: 0.8071
Test accuracy: 0.8071428537368774


In [ ]:
(train_imagesall, train_labels), (test_imagesall, test_labels) = mnist.load_data()

# train_imagesall=train_imagesall[:,:,:,np.newaxis]
# test_imagesall=test_imagesall[:,:,:,np.newaxis]
# train_images=([])
# test_images1=([])
# for i in range(0,60000):

#   #print(a.shape)
#   train_images.append(np.concatenate((train_imagesall[i],train_imagesall[i],train_imagesall[i]),axis=2))
# for i in range(0,10000):
#   test_images1.append(np.concatenate((test_imagesall[i],test_imagesall[i],test_imagesall[i]),axis=2))

train_images=train_imagesall.reshape(60000,28,28,1)
test_images1=test_imagesall.reshape(10000,28,28,1)

val_images = test_images1[0:2000,:,:]
val_labels = test_labels[0:2000]
test_images = test_images1[2000:,:,:]
test_labels = test_labels[2000:]


train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5
val_images = (val_images / 255) - 0.5



es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 8)
mc = ModelCheckpoint('/content/gdrive/MyDrive/models/task2_mnist_pretrain.h5', monitor='val_accuracy', mode='max', 
                     verbose=1, save_best_only=True)

# Train the model.
start_time = time.time()
history=saved_model.fit(
  train_images,
  to_categorical(train_labels),
  validation_data=(val_images, to_categorical(val_labels)),  
  epochs=5,
  batch_size=32,
  shuffle = True,
  callbacks=[es,mc],
)

scores = saved_model.evaluate(
  test_images,
  to_categorical(test_labels)
)

print('Test accuracy:', scores[1])
print("--- %s seconds ---" % (time.time() - start_time))

11493376/11490434 [==============================] - 0s 0us/step
Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2627 - accuracy: 0.9260 - val_loss: 0.1888 - val_accuracy: 0.9345

Epoch 00001: val_accuracy improved from -inf to 0.93450, saving model to /content/gdrive/MyDrive/models/task2_mnist_pretrain.h5
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0882 - accuracy: 0.9732 - val_loss: 0.0953 - val_accuracy: 0.9725

Epoch 00002: val_accuracy improved from 0.93450 to 0.97250, saving model to /content/gdrive/MyDrive/models/task2_mnist_pretrain.h5
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0642 - accuracy: 0.9805 - val_loss: 0.0718 - val_accuracy: 0.9760

Epoch 00003: val_accuracy improved from 0.97250 to 0.97600, saving model to /content/gdrive/MyDrive/models/task2_mnist_pretrain.h5
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0508 - accuracy: 0.9839 - val_loss: 